## 管理层信息抽取

In [ ]:
! cp /Users/alchemy_taotaox/Desktop/next/my_test/* ../corpus/

In [ ]:
! ls ../corpus

## 算法基本思路
1. 扫描pdf 获得页眉页脚
2. 扫描pdf 获得最小和最大的x坐标
    如果一个字符是最小的坐标，那么就和之前的行合并为一个段落
    如果一个字符是最大的坐标，那么就和之前的
2. 提取文档中的表格和字符串。

In [156]:
from pdfplumber.pdf import PDF
import re
from collections import Counter

def is_header(str, header):
    if header is None:
        return False
    return str == header

def is_foot(str, foot):
    if foot is None:
        return False
    
    s = re.sub(r'\s*\d+\s*', '#num#', str)
    s = s.strip()
    return s == foot or s == '#num#/#num#' or s== '#num#.'

def get_foot_header(pdf:PDF, check_pages_num=20, threshold = 0.9):
    head_counter = Counter()
    foot_counter = Counter()
    pages_num = len(pdf.pages)
    if pages_num < check_pages_num:
        check_pages_num = pages_num
    for i in range(check_pages_num):
        text_lines = pdf.pages[i].extract_words()
        if text_lines is not None and len(text_lines) == 0:
            continue
        top_line = text_lines[0]['text']
        bottom_line = text_lines[-1]['text']
        # print(bottom_line, " !!!!! ", top_line)
        bottom_line = re.sub(r'\d+', '#num#', bottom_line)
        head_counter.update([top_line])
        foot_counter.update([bottom_line])
    head, foot = "fadfasdfa", "dfadfadsfsdf"
    try:    
        most_common_head_word, count_head = head_counter.most_common(1)[0]
        most_common_foot_word, count_foot = foot_counter.most_common(1)[0]

   
        if count_head *1./check_pages_num >= threshold:
            head = most_common_head_word
        if count_foot *1./check_pages_num >= threshold:
            foot = most_common_foot_word
    except:
        pass
    return head, foot
    
    

In [157]:
import pdfplumber

In [158]:
from operator import itemgetter
import json

def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]



def get_lines_from_page(page, head, foot):
    tables = page.find_tables()
    table_bboxes = [i.bbox for i in tables]
    tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
    non_table_words = [word for word in page.extract_words() if not any(
        [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
    lines = []
    lines_x0_x1 =[]
    for cluster in pdfplumber.utils.cluster_objects(
        non_table_words + tables, itemgetter('top'), tolerance=5):
        if 'text' in cluster[0]:
            x_0 = x_1 = -1
            inner_lines = []
            for item in cluster:
                #if len(cluster) == 1 and (is_foot(item['text'], foot) or  is_header(item['text'], head)):
                #    continue
                # print(item, cluster)
                if x_0 == -1:
                    if 'x0' in item:
                        x_0 = item['x0']
                if 'x1' in item:
                    x_1 = item['x1']
                if 'text' in item:
                    inner_lines.append(item['text'])
                elif 'table' in item:
                    inner_lines.append("\n" + json.dumps(item['table'], ensure_ascii=False) + "\n")
            if len(inner_lines) > 0:
                lines.append(' '.join(inner_lines))
                lines_x0_x1.append((x_0, x_1))
                
            # lines.append(' '.join([i['text'] for i in cluster if not is_foot(i['text'], foot) and not is_header(i['text'], head)]))
        elif 'table' in cluster[0]:
            lines.append(cluster[0]['table'])
            lines_x0_x1.append((10000000, -1))

    # Find the minimum of the first elements
    if len(lines_x0_x1) == 0:
        return []
    min_first = min(x[0] for x in lines_x0_x1)

    # Find the maximum of the second elements
    max_second = max(x[1] for x in lines_x0_x1)
       
    # print(lines)
    
    # lines = lines[1:-1]
    if type(lines[0]) == str and is_header(lines[0], head):
        lines = lines[1:]
    if type(lines[-1]) == str and is_foot(lines[-1], foot):
        lines = lines[:-1]
    # return lines
    for index, item in enumerate(lines):
        # if type(lines[index]) == str:
        #     lines[index] = lines[index].strip()
        
        # 如果锁进， 那么要换行
        
        if abs(lines_x0_x1[index][0] - min_first) > 0.1 and lines_x0_x1[index][0] < 1000000:
            # print(lines_x0_x1[index][0], min_first, "----")
            if type(lines[index]) == str:
                lines[index] = "\n" + lines[index]
        # 如果没有写完，那么换行
        if abs(lines_x0_x1[index][1] - max_second) > 16 and lines_x0_x1[index][1] > 0:
            # print(lines_x0_x1[index][1], max_second, "----", lines[index])
            if type(lines[index]) == str:
                lines[index] = lines[index] + "\n"
        elif lines_x0_x1[index][1] > 0:
            # print('******FULL LINE******', lines_x0_x1[index][1], max_second, "----", lines[index])
            pass
        # print(lines[index], end='')
    return lines

In [159]:
filename = "../corpus/year_report_2.pdf"
# filename = "/tmp/b.PDF"
# # filename = "/tmp/c.PDF"
# # filename = "/tmp/e.PDF"
# filename = "/tmp/f.PDF"
filename = "../corpus/600271.pdf"
pdf = pdfplumber.open(filename)
# page = pdf.pages[10]
# page = pdf.pages[15]
page = pdf.pages[7]
# print(page.extract_text())
head, foot = get_foot_header(pdf)
print(head)
print(foot)
lines = get_lines_from_page(page, head, foot)
for line in lines:
    print("----", line, "----", type(line))

# print(pdf.pages[7].extract_words() )# 计算一个字符大概的宽度是多少，然后定义magic distance = 16
# print(pdf.pages[7].extract_text())

2022年年度报告
#num#
---- 
十二、 其他
 ---- <class 'str'>
---- □适用 √不适用
 ---- <class 'str'>
---- 第三节 管理层讨论与分析
 ---- <class 'str'>
---- 
一、经营情况讨论与分析
 ---- <class 'str'>
---- 2022年，面对严峻复杂的内外部形势，航天信息全体干部职工迎难而上、砥砺前行，统筹发
 ---- <class 'str'>
---- 
展和安全，党的建设坚强有力，发展战略更加清晰，研发创新成果丰硕，国企改革全面深化，市 ---- <class 'str'>
---- 场开拓取得突破，管理水平不断提高，发展质量稳步提升。公司圆满完成净利润、经济增加值等 ---- <class 'str'>
---- 年度主要经济指标，成本费用利润率同比提高1.03个百分点，资产负债率同比优化1.32个百分 ---- <class 'str'>
---- 点，营业收入利润率同比提高1.19个百分点。 ---- <class 'str'>
---- 1.以新战略牵引高质量发展行稳致远。
 ---- <class 'str'>
---- 
紧跟数字经济发展大潮，立足内外环境变化，在“十四五”既定规划基础上，系统研究提出
 ---- <class 'str'>
---- 
新战略。明确发展愿景是“成为世界一流的信息技术集团，致力于信息更安全、更增值，社会更 ---- <class 'str'>
---- 智慧、更诚信”，明确发展思路是“一二三五八”，明确发展目标是打造“战略新高度、创新新 ---- <class 'str'>
---- 速度、产业新维度、改革新动能、精神新面貌”的新航信。研究形成数字政府和企业数字化两个 ---- <class 'str'>
---- 产业规划和数字技术与产品体系规划，关键产业领域战略举措有序推进。 ---- <class 'str'>
---- 2.改革工作取得突破进展。
 ---- <class 'str'>
---- 
深入贯彻落实党中央、国务院系列部署，国企改革三年行动方案37项任务、集团公司改革考
 ---- <class 'str'>
---- 
核39项任务均按计划圆满完成。形成“科改示范行动”2022-2025年改革方

In [160]:
from pdfplumber.pdf import PDF
import json
def begin_mda(lines):
    len_lines = len(lines)
    if len_lines < 1:
        return False, -1
    # if type(lines[0]) != str:
    #     return False, -1
    # print(" check beign count ", len_lines)
    for index, item in enumerate(lines):
        # print("CHECK", item, index)
        if type(item) != str:
            continue
        if item.find('管理层讨论与分析') != -1 and item.find('第三节')!= -1 and item.find("...............") == -1 and item.find("请") == -1:
            # print(' BEGIN ', index)
            return True, index
    return False, -1
def end_mda(lines):
    if len(lines) < 1:
        return False, -1
    # if type(lines[0]) != str:
    #     return False, -1
    # print("check: ", lines[0], "$$$$$$")
    for index, item in enumerate(lines):
        if type(item) != str:
            continue
        if item.find('公司治理') != -1 and item.find('第四节')!= -1 and item.find("...............") == -1 and item.find("请") == -1:
            return True, index
    return False, -1

def get_all_lines_about_mda(pdf:PDF, head:str, foot:str):
    begin = False
    end = False
    all_lines = []
    index = 0
    # print(f"page number: {len(pdf.pages)}")
    for page in pdf.pages:
        index += 1
        if index < 5:
            continue
        # print(index, " ----- page ", head, foot)
        lines = get_lines_from_page(page, head, foot)
        # for line in lines:
        #     print(line)
        begin_mda_status, line_index = begin_mda(lines)
        # print(index, ' ---page   check ------', begin_mda_status)

        if begin is False and begin_mda_status:
            begin = True
            lines = lines[line_index:]
        end_mda_status, line_index = end_mda(lines)
        if begin and end_mda_status:
            
            all_lines.append(lines[:line_index])
            break
        if begin is True:
            all_lines.append(lines)
    # print(f" final page index {index}")
    content_list = []
    for lines in all_lines:
        for line_index, line in enumerate(lines):
            if type(line) != str:
                line = json.dumps(line, ensure_ascii=False)
            # print(line, end='')
            if line.strip() == '':
                continue
            content_list.append(line)
    return "".join(content_list)
            
content = get_all_lines_about_mda(pdf, head, foot)
if content is not None:
    print(content)
else:
    print("None")

第三节 管理层讨论与分析

一、经营情况讨论与分析
2022年，面对严峻复杂的内外部形势，航天信息全体干部职工迎难而上、砥砺前行，统筹发

展和安全，党的建设坚强有力，发展战略更加清晰，研发创新成果丰硕，国企改革全面深化，市场开拓取得突破，管理水平不断提高，发展质量稳步提升。公司圆满完成净利润、经济增加值等年度主要经济指标，成本费用利润率同比提高1.03个百分点，资产负债率同比优化1.32个百分点，营业收入利润率同比提高1.19个百分点。1.以新战略牵引高质量发展行稳致远。

紧跟数字经济发展大潮，立足内外环境变化，在“十四五”既定规划基础上，系统研究提出

新战略。明确发展愿景是“成为世界一流的信息技术集团，致力于信息更安全、更增值，社会更智慧、更诚信”，明确发展思路是“一二三五八”，明确发展目标是打造“战略新高度、创新新速度、产业新维度、改革新动能、精神新面貌”的新航信。研究形成数字政府和企业数字化两个产业规划和数字技术与产品体系规划，关键产业领域战略举措有序推进。2.改革工作取得突破进展。

深入贯彻落实党中央、国务院系列部署，国企改革三年行动方案37项任务、集团公司改革考

核39项任务均按计划圆满完成。形成“科改示范行动”2022-2025年改革方案，接续推进五大类55项任务。连续两年获评国资委标杆“科改示范企业”，改革案例入选国资委国企改革三年行动案例集。作为全国十个优秀课程之一，入选国资委“科改示范”培训班专题课程。改革实践项目获评2022中国企业改革发展优秀成果和国防企协年度管理创新成果。坚持收益共享、风险共担，新增云享科技、广西航信、天津航信等单位实施中长期激励改革，累计已有6家单位应用股权激励、员工持股、岗位分红和超额利润分享等4类科改政策，显著提升干部职工干事创业积极性。抢抓数字新疆产业发展重大机遇，完成参股投资“新疆信息产业集团有限公司”论证并报集团公司审批。结合数字产业发展需要调整组织架构，重组“3+1”产业板块。组建加密经济事业部，优化信息中心等专业支撑机构设置，强化业务协同与资源共享。3.高效科技创新体系初步构建。

着力打造适应转型升级需要的高效科技创新体系，涵盖技术和产品、研发组织、专业技术人

才、管理体系及基础支撑等5个子体系，明确到2025年成为国家“科技型骨干企业”发展目标。结合重大创新工程，梳理形成核心技术与关键产品清单，

## 抓取信息



In [161]:
from datetime import datetime, timedelta

def split_interval(start, end, day_cnt):
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start, date_format)
    end_date = datetime.strptime(end, date_format)
    
    result = []

    while start_date < end_date:
        # Calculate the end date of the current interval.
        interval_end_date = start_date + timedelta(days=day_cnt-1)

        # Ensure that the interval doesn't go beyond the overall end date.
        if interval_end_date > end_date:
            interval_end_date = end_date

        # Append the current interval to the result list.
        result.append(
            (start_date.strftime(date_format), 
             interval_end_date.strftime(date_format), 
             (interval_end_date - start_date).days + 1)
        )

        # Move the start date to the next day after the current interval.
        start_date = interval_end_date + timedelta(days=1)

    return result
results = split_interval('2023-01-01', '2023-07-31', 1)

print(results)

[('2023-01-01', '2023-01-01', 1), ('2023-01-02', '2023-01-02', 1), ('2023-01-03', '2023-01-03', 1), ('2023-01-04', '2023-01-04', 1), ('2023-01-05', '2023-01-05', 1), ('2023-01-06', '2023-01-06', 1), ('2023-01-07', '2023-01-07', 1), ('2023-01-08', '2023-01-08', 1), ('2023-01-09', '2023-01-09', 1), ('2023-01-10', '2023-01-10', 1), ('2023-01-11', '2023-01-11', 1), ('2023-01-12', '2023-01-12', 1), ('2023-01-13', '2023-01-13', 1), ('2023-01-14', '2023-01-14', 1), ('2023-01-15', '2023-01-15', 1), ('2023-01-16', '2023-01-16', 1), ('2023-01-17', '2023-01-17', 1), ('2023-01-18', '2023-01-18', 1), ('2023-01-19', '2023-01-19', 1), ('2023-01-20', '2023-01-20', 1), ('2023-01-21', '2023-01-21', 1), ('2023-01-22', '2023-01-22', 1), ('2023-01-23', '2023-01-23', 1), ('2023-01-24', '2023-01-24', 1), ('2023-01-25', '2023-01-25', 1), ('2023-01-26', '2023-01-26', 1), ('2023-01-27', '2023-01-27', 1), ('2023-01-28', '2023-01-28', 1), ('2023-01-29', '2023-01-29', 1), ('2023-01-30', '2023-01-30', 1), ('2023-01

In [ ]:
import requests
import time
from tqdm import tqdm
import json




def request_data(page_size=1000, page_num=1, se_date="2023-04-26~2023-04-26"):
    url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"
    if page_num > 1:
        url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"

    params = {
        "pageNum": page_num,
        "pageSize": page_size,
        "column": "szse",
        "tabName": "fulltext",
        "plate": "",
        "stock": "",
        "searchkey": "",
        "secid": "",
        "category": "category_ndbg_szsh",
        "trade": "",
        "seDate": se_date,
        "sortName": "",
        "sortType": "",
        "isHLtitle": "true"
    }

    response = requests.post(url, data=params)

    if response.status_code == 200:
        return response.json()  # 或者返回response.text取决于你需要什么样的数据格式
    else:
        return None

file = open('report_meta_info.txt', 'w')
results = split_interval('2023-01-01', '2023-07-31', 1)
results = results[::-1]
#for date_range in results:
for i, date_range in enumerate(tqdm(results)):
    se_date = f"{date_range[0]}~{date_range[1]}"
    
    page_size = 30
    data = request_data(page_size=10, se_date=se_date)
    try_times = 0
    while data is None:
        time.sleep(1)
        data = request_data(page_size=10, se_date=se_date)
        try_times += 1
        print(f"craw data wrong, try {try_times}")
        if try_times == 3:
            break  
    if data is None:
        continue
    page_count = (data['totalAnnouncement'] -1 )// page_size  + 1
    print(page_count)
    begin = time.time()
    for index in range(1, page_count+1):
    
        data = request_data(page_num=index, page_size=page_size, se_date=se_date)
        try_times = 0
        while data is None:
        
            time.sleep(3)
            data = request_data(page_num=index, page_size=page_size, se_date=se_date)
            try_times += 1
            print(f"craw data wrong, try {try_times}")
            if try_times == 3:
                break
        
        last = time.time() - begin
        need_time = last/index * page_count  - last
        every_time = last*1./index
        print(f"{index}/{page_count}: last time {last} 秒， need {need_time}, every page cost: {every_time}, count: { len(data['announcements'])}, but page size is {page_size}")
        for item in data['announcements']:
            file.write(json.dumps(item, ensure_ascii=False) + "\n")
file.close()
    

In [162]:
from urllib.parse import urljoin
import os
from tqdm import tqdm
import requests
import json

def download_and_process_file(url, local_path='/tmp/a.PDF', try_count=3):
    try_times = 0
    response = None
    while True:
        # 下载文件
        response = requests.get(url, proxies={})
        if response.status_code == 200:
            break
            # return response.json()  # 或者返回response.text取决于你需要什么样的数据格式
        else:
            response = None
        try_times += 1
        if try_times >= try_count:
            time.sleep(3)
            break
    if response is None:
        return response
    # response.raise_for_status()  # 确保请求成功

    # 将文件写入本地
    with open(local_path, 'wb') as output_file:
        output_file.write(response.content)

    # 读取PDF文件的内容
    
    pdf = pdfplumber.open(local_path)
    head, foot = get_foot_header(pdf)
    content = get_all_lines_about_mda(pdf, head, foot)
    #print(content)
    # 删除临时文件
    os.remove(local_path)

    return content
'''
1. read meta file
2. download pdf file into tmp file
3. get the necessary content
4. write data into file
   {'stock_code', 'stock_name', 'content':   'url':  'pubtime': xxx}

'''
import os
if 'all_proxy' in os.environ:
    print("pop all proxy")
    os.environ.pop('all_proxy')
base_url = 'http://static.cninfo.com.cn/'
meta_file = 'report_meta_info.txt'
data_file = 'report_data_info.txt'
file = open(data_file, "w")
lines = open(meta_file).readlines()
code_set = set()
for index, line in tqdm(enumerate(lines)):
    # if index < 60:
    #     continue
    info = json.loads(line.strip())
    '''
    "secCode": "001211", "secName": "双枪科技, adjunctUrl
    '''
    title = info['announcementTitle']
    if title.find('关于') != -1 or title.find("摘要") != -1 or title.find('2022') == -1 or title.find("英文") != -1:
        continue
    stock_code = info['secCode']
    if stock_code in code_set:
        continue
    # code_set.add(stock_code)
    stock_name = info['secName']
    relative_url = info['adjunctUrl']
    pub_time = relative_url.split('/')[-2]
    url =  urljoin(base_url, relative_url)
    content = download_and_process_file(url, "/tmp/a.PDF")
    if content is None or content == "":
        pass
    else:
        code_set.add(stock_code)
    result = {"title": title, 'pubtime': pub_time, 'stock_code': stock_code, 'stock_name': stock_name, 'content': content, 'url': url}
    file.write(json.dumps(result, ensure_ascii=False) + "\n")
    # if index == 100:
    #     break

   
   

73it [10:26,  8.58s/it]


KeyboardInterrupt: 

In [89]:
import multiprocessing

def process_item(args):
    
    item, lock = args
    info = json.loads(item.strip())
    '''
    "secCode": "001211", "secName": "双枪科技, adjunctUrl
    '''
    title = info['announcementTitle']
    if title.find('关于') != -1 or title.find("摘要") != -1 or title.find('2022') == -1 or title.find("英文") != -1:
        return
    stock_code = info['secCode']
    # with lock:
        # if stock_code in code_set:
            # return
    # code_set.add(stock_code)
    stock_name = info['secName']
    relative_url = info['adjunctUrl']
    pub_time = relative_url.split('/')[-2]
    url =  urljoin(base_url, relative_url)
    content = download_and_process_file(url, "/tmp/a.PDF")
    if content is None or content == "":
        pass
    else:
        pass
        # with lock:
            # code_set.add(stock_code)
    result = {"title": title, 'pubtime': pub_time, 'stock_code': stock_code, 'stock_name': stock_name, 'content': content, 'url': url}
    with lock:
        with open("output.txt", "a") as file:
            # file.write(str(result) + "\n")
            file.write(json.dumps(result, ensure_ascii=False) + "\n")


# 定义要处理的数组
data_list = lines  # 请替换为实际的数组元素
new_data_list = []
stock_code_set = set()
for item in data_list:
    info = json.loads(item.strip())
    '''
    "secCode": "001211", "secName": "双枪科技, adjunctUrl
    '''
    title = info['announcementTitle']
    if title.find('关于') != -1 or title.find("摘要") != -1 or title.find('2022') == -1 or title.find("英文") != -1:
        continue
    stock_code = info['secCode']
    if stock_code in stock_code_set:
        continue
    stock_code_set.add(stock_code)
    new_data_list.append(item)

    
print(len(new_data_list), " ----- ")
# 定义要开启的进程数量
n = 6  # 请替换为实际需要的进程数量

# 创建进程池
pool = multiprocessing.Pool(processes=n)

# 创建进程锁和共享的集合
with multiprocessing.Manager() as manager:
    lock = manager.Lock()
    # processed_set = manager.set()

    # 使用进程池并行处理数组中的每个元素
    pool.map(process_item, [(item, lock) for item in new_data_list[:100]])

# 关闭进程池
pool.close()
pool.join()


5126  ----- 


TypeError: process_item() missing 1 required positional argument: 'lock'

In [145]:
filename = "../corpus/year_report_2.pdf"
filename = "/tmp/b.PDF"
# filename = "/tmp/c.PDF"
# filename = "/tmp/e.PDF"
filename = "../corpus/kg_expert.pdf"
pdf = pdfplumber.open(filename)
# page = pdf.pages[10]
# page = pdf.pages[15]
head, foot = get_foot_header(pdf)
print(head)
print(foot)
for id in range(144, 152):
    print(id, "------------")
    page = pdf.pages[id]
    # print(page.extract_text())


    lines = get_lines_from_page(page, head, foot)
    for line in lines:
        print(line, end='')

# print(pdf.pages[7].extract_words() )# 计算一个字符大概的宽度是多少，然后定义magic distance = 16
# print(pdf.pages[7].extract_text())

fadfasdfa
dfadfadsfsdf
144 ------------
非财务篇 - 管理层讨论和分析125.什么是管理层讨论与分析？一般包括哪些内容？管理层讨论与分析是上市公司定期报告的重要组成部分。它通过对公司财务报表的相关财务数据的文字解读，对公司经营中固有的风险和不确定性的提示，向投资者揭示公司管理层对于公司过去（报告期内及临近期间）经营状况的评价分析以及对公司未来发展趋势和发展前景的前瞻性判断与预期。监管部门要求上市公司编制并披露管理层讨论与分析的目的，是使公众投资者能够有机会定期了解上市公司管理层自身对公司财务状况与经营成果的分析评价以及公司未来一定期间的发展战略和具体规划。这些信息在财务报表及附注中并没有得到充分揭示，但是对投资者进行相关投资决策却是至关重要的内容。管理层讨论与分析通常包括报告期内公司所处行业情况，从事的业务情况，核心竞争力的重要变化及对公司所产生的影响，主要经营业务、资产及负债状况、未盈利企业尚未盈利的原因及影响、投资状况、重大资产和股权144145 ------------
146 ------------
一方面，在行业情况部分，可以关注公司介绍的所处行业基本情况、发展阶段、周期性特点以及公司所处的行业地位情况，重点关注报告期内前述事项发生的重大变化。公司还会分析新公布的法律、行政法规、部门规章、行业政策对所处行业的重大影响。同时，创业板公司还应当结合所属行业的特点，针对性披露技术、产业、业态、模式等能够反映行业竞争力的信息。科创板公司还应当结合所属行业的特点，针对性披露科研水平、科研人员、科研投入等能够反映行业竞争力的信息。另一方面，在风险因素部分，可以关注公司对于行业格局和趋势的分析和展望。公司需结合自身的业务规模、经营区域、产品类别以及竞争对手等情况，介绍与公司业务关联的宏观经济层面或行业环境层面的发展趋势，以及公司的行业地位或区域市场地位的变动趋势。同时，结合主要业务的市场变化情况、营业成本构成的变化情况、市场份额变化情况等因素，分析公司的主要行业优势和劣势，并说明变化对公司未来经营业绩和盈利能力的影响。128.如何通过管理层讨论与分析认识企业的核心竞争力？公司的核心竞争力包括核心管理团队、关键技术人员、专有设备、专利、非专利技术、特许经营权、土地使用权、水面养殖权、探矿权、采矿权、独特经营方式和盈利模式、允许他人